# Pre-Requisite Installations

Please do the following steps initially:
- Install the AWS CLI from: https://aws.amazon.com/cli/
- Open a command line and type `aws configure`
- Leave the default region blank, and enter your AWS access id and secret key when prompted.

**(NOTE: Requires python >= 3.8)**

Dependencies (run the command in the next cell if you need these dependencies installed):
- boto3
- moviepy
- wget (not needed)
- intervaltree

In [69]:
# # Run these commands to install dependencies
# %pip install boto3
# %pip install moviepy
# %pip install wget (not needed)
# %pip install intervaltree

# Import everything that is needed

In [70]:
# General imports
import os
import json
import shutil
import random
import regex as re
import warnings
import uuid

# Video clipping
import datetime
from moviepy.editor import *
import subprocess
import intervaltree

# Setup of ego4d downloads
from zipfile import ZipFile
import requests
from io import BytesIO

# Custom classes
from utils import cd

In [71]:
# Setup and Data paths that are defined here
PROFILE_NAME = '' # NOTE: If you have a profile name for AWS cli, enter here
DATA_DIR = 'data'
EGO4D_SETUP_DIR = 'ego4d_setup'
VIDEO_PATH = os.path.join(DATA_DIR, 'videos')
ANNOTATION_PATH = os.path.join(DATA_DIR, 'annotations')
TRAINING_PATH = os.path.join(VIDEO_PATH, 'training')
TESTING_PATH = os.path.join(VIDEO_PATH, 'testing')
VALIDATION_PATH = os.path.join(VIDEO_PATH, 'validation')

# Download and Prepare Dataset

In [72]:
# Prepare directories and setup
def perform_setup(delete_annotations=False, delete_clips=False, delete_setup=False, delete_data=False):
    # If you want to delete already existing folders and contents
    if delete_setup and os.path.exists(EGO4D_SETUP_DIR): shutil.rmtree(EGO4D_SETUP_DIR)
    if delete_clips and os.path.exists(VIDEO_PATH): shutil.rmtree(VIDEO_PATH)
    if delete_annotations and os.path.exists(ANNOTATION_PATH): shutil.rmtree(ANNOTATION_PATH)
    if delete_data and os.path.exists(DATA_DIR): shutil.rmtree(DATA_DIR)

    # If any dir doesn't exist, make it
    if not os.path.exists(EGO4D_SETUP_DIR): os.makedirs(EGO4D_SETUP_DIR)
    if not os.path.exists(ANNOTATION_PATH): os.makedirs(ANNOTATION_PATH)
    if not os.path.exists(VIDEO_PATH): os.makedirs(VIDEO_PATH)
    if not os.path.exists(TRAINING_PATH): os.makedirs(TRAINING_PATH)
    if not os.path.exists(TESTING_PATH): os.makedirs(TESTING_PATH)
    if not os.path.exists(VALIDATION_PATH): os.makedirs(VALIDATION_PATH)

# Uncomment to setup the dataset
# delete_annotations: resets data/annotations 
# delete_clips: resets data/videos
# delete_setup: resets ego4d_setup directory (should be deleted normally anyway)
# delete_data: deletes all of data directory and its subdirectories
perform_setup()

> Retrieve ego4d data retrieval tool

In [73]:
# Retrieve the zip file for setup
print('Downloading the setup zip file..')

#Defining the zip file URL
url = 'https://github.com/facebookresearch/Ego4d/archive/refs/heads/main.zip'
# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)

# extracting the zip file contents
zipfile= ZipFile(BytesIO(req.content))
zipfile.extractall(EGO4D_SETUP_DIR)
print('Finished retrieving the setup files.')

Finished retrieving the setup files.


In [74]:
# Setting up the annotations and retrieving it
def setup_annotations(annotation_regex):
    # Check if annotations are already present
    if os.path.exists(ANNOTATION_PATH):
        files = next(os.walk(ANNOTATION_PATH), (None, None, []))[2]
        rgx = re.compile(annotation_regex)
        newlist = list(filter(rgx.match, files))

        # If not present, then download the annotations and save them
        if not newlist:
            print('Downloading annotations..')

            # Command to download ALL annotations
            if PROFILE_NAME:
                command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=ego4d_data', '--datasets', 'annotations', '--aws_PROFILE_NAME', PROFILE_NAME, '--yes']
            else:
                command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=ego4d_data', '--datasets', 'annotations', '--yes']
            with cd(os.path.join(EGO4D_SETUP_DIR, "Ego4d-main")):
                subprocess.run(command, check=True)
            # Copy and save only needed annotations in the data directory
            annotation_dir = os.path.join(EGO4D_SETUP_DIR, "Ego4d-main", "ego4d_data", "v1", "annotations")
            for file in os.listdir(annotation_dir):
                if "moments" in file: # TODO: Should be changed later to not have string
                    shutil.copy(os.path.join(annotation_dir, file), os.path.join(ANNOTATION_PATH))
            print('Finished downloading and setting up annotations.')
        else:
            print('Annotations already present!')
        
# Setting up the annotations
# annnotation_regex: regular expression of the files you want to copy from ALL annotations
setup_annotations("moments*")

Annotations already present!


# Pre-Process

### Process moments_train.json
> Process moments_train.json and filter videos with given classes.

In [75]:
# The classes to be used in this project
classes = [
    "browse_through_clothing_items_on_rack_/_shelf_/_hanger",
    "read_a_book_/_magazine_/_shopping_list_etc.", 
    "\"cut_/_chop_/_slice_a_vegetable,_fruit,_or_meat\"",
    "throw_away_trash_/_put_trash_in_trash_can",
    "clean_/_wipe_other_surface_or_object",
    "dig_or_till_the_soil_with_a_hoe_or_other_tool",
    "wash_dishes_/_utensils_/_bakeware_etc."
    ]
# Training, testing, and validation annotation files
# Filtered clip uids dictionary containing only classes listed above
filtered_dict = {
    'training': {
        'file_name': 'moments_train.json',
        'count': 200,
        'data': {}
    },
    'testing': {
        'file_name': 'moments_val.json',
        'count': 20,
        'data': {}
    },
    'validation': {
        'file_name': 'moments_val.json',
        'count': 20,
        'data': {}
    }
}

# Setup pre-filtered dictionary
for data_type in filtered_dict:
    with open(os.path.join(ANNOTATION_PATH, filtered_dict[data_type]['file_name']), 'r') as json_file:
        # Load the annotation file
        loaded_json = json.load(json_file)

        # For each video and for each clips within the video
        # retrieve clip_uid, video_uids, label, start_time, and end_time
        for video in loaded_json['videos']:
            for clip in video['clips']:
                # Setup each clip information
                filtered_dict[data_type]['data'][clip["clip_uid"]] = {
                    "video_uid": video["video_uid"],
                    "annotations": {}
                }
                # For each annotation, retrieve annotator id and annotations
                for annotation in clip["annotations"]:
                    for label_item in annotation["labels"]:
                        label = label_item["label"]
                        if label in classes and (label_item["end_time"] - label_item["start_time"] >= 5):
                            if label in filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"]:
                                filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"][label].append(
                                    (label_item["start_time"], label_item["end_time"])
                                )
                            else:
                                filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"][label] = [
                                    (label_item["start_time"], label_item["end_time"])
                                ]
                # Same as before, filter out empty annotations
                if not filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"]:
                    del filtered_dict[data_type]['data'][clip["clip_uid"]]

# Do final processing with the filtered dictionary
count = 0
full_filtered_dict = {}
for data_type, data_info in filtered_dict.items():
    full_filtered_dict[data_type] = {
        "file_name": data_info["file_name"],
        "count": data_info["count"],
        "data": {}
    }
    for clip_uid, clip_info in data_info["data"].items():
        full_filtered_dict[data_type]["data"][clip_uid] = {
            "video_uid": clip_info["video_uid"],
            "annotations": {}
        }
        for label in clip_info["annotations"]:
            tree = intervaltree.IntervalTree.from_tuples(clip_info["annotations"][label])
            tree.merge_overlaps(strict=False)
            full_filtered_dict[data_type]["data"][clip_uid]["annotations"][label] = sorted(tree.iter())

print(len(full_filtered_dict))
with open('test.json', 'w') as jp:
    json.dump(full_filtered_dict, jp, indent=4)

3


### Trim videos into clips
> Using moviepy, split videos into clips

In [76]:
# Function to cut/trim the video to desired lengths and save it
def cut_video(VIDEO_PATH, times, save_path):
    # Load the video once and close using context manager
    with VideoFileClip(VIDEO_PATH, target_resolution=(224,224), resize_algorithm="fast_bilinear", audio=False) as vid_clip:
        for start_time, end_time, _ in times:
            # Segment information and random filename generation
            start_time = start_time
            end_time = end_time
            fileName = f"{uuid.uuid1()}.mp4"

            # Convert the amount of seconds in time xx:xx:xx.xxxxxx
            start_time = str(datetime.timedelta(seconds = start_time))
            end_time = str(datetime.timedelta(seconds = end_time))

            # Clip the video and save it to the specified path
            clip = vid_clip.subclip(start_time, end_time)
            if not os.path.exists(save_path): os.makedirs(save_path)
            clip.write_videofile(os.path.join(save_path, fileName), threads=8, logger=None, preset="ultrafast")
            clip.close()

In [77]:
# %%capture cap --no-stderr
# ^ To save the output to external file because it's waaaaay too large to display in notebook
# Function to retrieve a sample of clips for each class
all_clips_dict = {
    'training': {},
    'testing': {},
    'validation': {}
}

def generate_clip_uids(filtered_annotations):
    # Seperate clip uids by their class
    for clip_type in all_clips_dict:
        for clip_uid, clip_info in filtered_annotations[clip_type]['data'].items():
            for annotation in clip_info["annotations"]:
                if annotation in all_clips_dict[clip_type]:
                    all_clips_dict[clip_type][annotation].append(clip_uid)
                else:
                    all_clips_dict[clip_type][annotation] = [clip_uid]
        # Grab random sample of num_clips for each class
        for annotation in all_clips_dict[clip_type]:
            maximum = min(len(all_clips_dict[clip_type][annotation]), filtered_annotations[clip_type]['count'])
            print(f'Generated samples for {annotation} in {clip_type} with total of {maximum}.')
            all_clips_dict[clip_type][annotation] = random.sample(all_clips_dict[clip_type][annotation], maximum)

# Function to download and setup the clips 
def setup_clips(filtered_annotations, save_path, overwrite=False):
    # Check if the clips already exist if we are not overwriting
    if not overwrite:
        dirs = next(os.walk(VIDEO_PATH), (None, None, []))[1]
        for dir in dirs:
            subdirs = next(os.walk(VIDEO_PATH, dir), (None, None, []))[2]
            for subdir in subdirs:
                files = next(os.walk(os.path.join(VIDEO_PATH, dir, subdir)), (None, None, []))[2]
                if dir.replace(";", "/") in all_clips_dict['training'] and files:
                    warnings.warn("Clips may already exist. If you want to redo the process, set the `overwrite` argument to true while calling the function.")
                    return

    # Get all the randomly generated clip samples and go through them
    generate_clip_uids(filtered_annotations)

    # Go through each type of data (training, testing, validation)
    for data_type in filtered_annotations:
        for cls, clip_uids in all_clips_dict[data_type].items():
            useable_name = cls.replace("/", ";") # Paths can't have / in their name so, replace it

            # Make sure the required path exists without old items
            if os.path.exists(os.path.join(VIDEO_PATH, data_type, useable_name)): shutil.rmtree(os.path.join(VIDEO_PATH, data_type, useable_name))
            os.makedirs(os.path.join(VIDEO_PATH, data_type, useable_name))

            print(f"Processing clips for class: {cls}")
            for clip_uid in clip_uids:
                # Command to retrieve clip by its uid
                if PROFILE_NAME:
                    command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=temp_clip', '--datasets', 'clips', '--video_uids', clip_uid, '--aws_PROFILE_NAME', PROFILE_NAME, '--yes']
                else:
                    command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=temp_clip', '--datasets', 'clips', '--video_uids', clip_uid, '--yes']
                # Path to the downloaded clip
                full_vid_path = os.path.join(EGO4D_SETUP_DIR, "Ego4d-main", "temp_clip", "v1", "clips")
                
                # Use context manager to ensure any change in directory is returned to original
                with cd(os.path.join(EGO4D_SETUP_DIR, "Ego4d-main")):
                    # Download the video
                    subprocess.run(command, check=True)
                    
                # Get the clip information and trim the video
                clip_info = filtered_annotations[data_type]['data'][clip_uid]
                for label in clip_info['annotations']:
                    folder_label = label.replace("/", ";")
                    cut_video(os.path.join(full_vid_path, f"{clip_uid}.mp4"), clip_info['annotations'][label], os.path.join(VIDEO_PATH, data_type, folder_label))
                
                # Clean up after for the specific clip
                shutil.rmtree(os.path.join(EGO4D_SETUP_DIR, "Ego4d-main", "temp_clip", "v1"))
    
    # Clean up the overall setup directory
    shutil.rmtree(EGO4D_SETUP_DIR)

# Download and setup clips
# filtered_dict: previously filtered dictionary of clips
# VIDEO_PATH: path to save the clips
setup_clips(full_filtered_dict, VIDEO_PATH)